# **Neck Posture with video detection**

In [2]:
import cv2
print(cv2.__version__)

4.10.0


In [3]:
import os
import numpy as np
import mediapipe as mp
import pandas as pd

In [7]:
# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Paths to dataset folders
good_posture_dir = '/Users/gabrielaclementedeoliveira/Documents/GitHub/final_project_ironhak/pics_ml/good_neck_posture'
bad_posture_dir = '/Users/gabrielaclementedeoliveira/Documents/GitHub/final_project_ironhak/pics_ml/bad_neck_posture'

I0000 00:00:1723452420.362593  351839 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M2
W0000 00:00:1723452420.438867  359506 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1723452420.446935  359507 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [8]:
# Function to extract key points
def extract_keypoints(landmarks):
    keypoints = []
    for landmark in landmarks.landmark:
        keypoints.extend([landmark.x, landmark.y, landmark.z])
    return keypoints

# Prepare a list to hold the dataset
data = []

# Process images in the "good_neck_posture" folder
for img_name in os.listdir(good_posture_dir):
    img_path = os.path.join(good_posture_dir, img_name)
    image = cv2.imread(img_path)

    # Convert image to RGB for MediaPipe
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image to extract pose landmarks
    result = pose.process(image_rgb)

    if result.pose_landmarks:
        # Extract key points
        keypoints = extract_keypoints(result.pose_landmarks)
        
        # Append label ("good" = 0)
        keypoints.append(0)
        
        # Add to the dataset list
        data.append(keypoints)

# Process images in the "bad_neck_posture" folder
for img_name in os.listdir(bad_posture_dir):
    img_path = os.path.join(bad_posture_dir, img_name)
    image = cv2.imread(img_path)

    # Convert image to RGB for MediaPipe
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image to extract pose landmarks
    result = pose.process(image_rgb)

    if result.pose_landmarks:
        # Extract key points
        keypoints = extract_keypoints(result.pose_landmarks)
        
        # Append label ("bad" = 1)
        keypoints.append(1)
        
        # Add to the dataset list
        data.append(keypoints)

# Convert the dataset list to a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('neck_posture_dataset.csv', index=False)

/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


## Video recognition of the neck posture

In [ ]:
# Initialize MediaPipe Pose
mp_pose_video = mp.solutions.pose
pose_video = mp_pose_video.Pose()

# Initialize OpenCV webcam
cap = cv2.VideoCapture(0)

# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose_video.process(rgb_frame)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        
        # Extract landmarks for the relevant points
        nose = [landmarks[mp_pose_video.PoseLandmark.NOSE.value].x,
                landmarks[mp_pose_video.PoseLandmark.NOSE.value].y]
        left_shoulder = [landmarks[mp_pose_video.PoseLandmark.LEFT_SHOULDER.value].x,
                         landmarks[mp_pose_video.PoseLandmark.LEFT_SHOULDER.value].y]
        right_shoulder = [landmarks[mp_pose_video.PoseLandmark.RIGHT_SHOULDER.value].x,
                          landmarks[mp_pose_video.PoseLandmark.RIGHT_SHOULDER.value].y]
        
        # Midpoint of the shoulders
        mid_shoulder = np.mean([left_shoulder, right_shoulder], axis=0)

        # Calculate the angle between the nose and shoulder line
        head_to_shoulder_angle = calculate_angle(nose, mid_shoulder, [mid_shoulder[0], mid_shoulder[1] - 0.1])

        # Display the angle and classify posture
        if head_to_shoulder_angle < 150:  # You can adjust this threshold based on experimentation
            cv2.putText(frame, 'Forward Head Posture Detected', (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        else:
            cv2.putText(frame, 'Good Posture', (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Draw landmarks
        mp.solutions.drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp_pose_video.POSE_CONNECTIONS)

    # Display the result
    cv2.imshow('Posture Detection', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1722957278.082738 224632293 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1722957278.143031 224752241 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1722957278.150424 224752241 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/opt/anaconda3/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
